# Gemma 3 1B

Revisión informal de la arquitectura del modelo Gemma 3 1B.

Modelo de código abierto publicado por Google en marzo de 2025.

- [https://storage.googleapis.com/deepmind-media/gemma/Gemma3Report.pdf](https://storage.googleapis.com/deepmind-media/gemma/Gemma3Report.pdf)

Mientras que la serie de modelos Gemma 3 es multimodal, el modelo 1B sólo soporta texto.

## 1. Arquitectura

El modelo se basa en la arquitectura _Transformer_ que procesa una secuencia de entrada y genera una secuencia de salida.

```
Input ─> Output
```

### 1.1. Decoder-Only

El modelo sólo implementa el _decoder_, prescidiendo del _encoder_ presente en la arquitectura clásica de los _transformers_.

```
Input ─> Decoder ─> Output
```

### 1.2. Decoder Stack

El modelo utiliza un _stack_ de 26 _decoders_.

```
Input ─> 26 x Decoder ─> Output
```

La salida del primer _decoder_ alimenta al segundo _decoder_, la salida del segundo _decoder_ alimenta al tercer _decoder_, y así sucesivamente.

## 2. Input

El texto de entrada se procesa por un _tokenizer_, y a la entrada del modelo se aplica una capa de _embedding_.

```
Input ─> Embedding ─> 26 x Decoder ─> Output
```

### 2.1. Tokenizer

El modelo utiliza _SentencePiece_ como _tokenizer_.

- [https://github.com/google/sentencepiece](https://github.com/google/sentencepiece)

La entrada del _tokenizer_ es texto libre de tamaño arbitrario, y su salida es una secuencia $T$ de _tokens_ de longitud $n$.

Trata el texto como una secuencia de caracteres Unicode, de forma que no depende de espacios o delimitadores específicos de cada idioma para segmentarlo en _tokens_. Los _tokens_ se gestionan internamente como secuencias de _bytes_, técnica conocida como _byte-pair-encoding (BPE)_, por lo que puede procesar cualquier carácter o secuencia de caracteres aunque sean desconocidos.

Los espacios en blanco se sustituyen por el carácter especial "▁" (_U+2581_), que se une al inicio de la palabra siguiente, y se tratan como _tokens_, de forma que se preserva la estructura del texto original. Por su parte, los dígitos numéricos se tratan como _tokens_ individuales, de forma que el número "123" se trata como tres _tokens_ individuales.

El _tokenizer_ entrenado para el modelo se llama `Gemma3Tokenizer` y tiene un vocabulario de 256.128 entradas. Entre otros, tiene los clásicos _tokens_ de inicio (_BOS_) y final de secuencia (_EOS_), _padding_ (_PAD_), desconocido (_UNK_), inicio (_START_OF_TURN_) y final (_END_OF_TURN_) de turno de conversación cuando existen múltiples participantes, y los relativos a la codificación de imágenes que en la versión 1B no se utilizan.

Dimensiones:

- $ T \in \{0, 1, \dots, V-1\}^{1 \times n} $

### 2.2. Embedding

La primera capa del modelo es una capa de _embedding_ de 1.152 dimensiones.

Su matriz de pesos $W$ es inicializada con una distribución normal y actualizada durante el entrenamiento.

- $ W = \begin{bmatrix}
w_{1,1} & w_{1,2} & \ldots & w_{1,1152}
\\ w_{2,1} & w_{2,2} & \ldots & w_{2,1152}
\\ \vdots & \vdots & \ddots & \vdots
\\ w_{256128,1} & w_{256128,2} & \ldots & w_{256128,1152}
\end{bmatrix} $

La salida de esta capa se obtiene al indexar la matriz de pesos $W$ con cada _token_ de la secuencia de _tokens_ $T$.

- $ \text{EMB} = \begin{bmatrix}
W[T_1]
\\ W[T_2]
\\ \vdots
\\ W[T_n]
\end{bmatrix} = \begin{bmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,1152}
\\ x_{2,1} & x_{2,2} & \ldots & x_{2,1152}
\\ \vdots & \vdots & \ddots & \vdots
\\ x_{n,1} & x_{n,2} & \ldots & x_{n,1152}
\end{bmatrix} $

Siguiendo el _paper_ original de la arquitectura _Transformer_, la salida se multiplica por la raíz cuadrada del número de dimensiones.

- $ X = \text{EMB} \sqrt{1152} $

La secuencia de _embeddings_ $X$ se propaga por la red sometiéndola a transformaciones funcionales a medida que avanza por las distintas capas de la red.

Dimensiones:

- $ T \in \{0, 1, \dots, V-1\}^{1 \times n} $

- $ W \in \mathbb{R}^{256128 \times 1152} $

- $ \text{EMB} \in \mathbb{R}^{n \times 1152} $

- $ X \in \mathbb{R}^{n \times 1152} $

Huelga decir que el modelo funciona con _batches_, por lo que las dimensiones reales son (_tamaño de batch_, _longitud de secuencia_, _dimensiones de los embeddings_), pero por simplicidad en el análisis se omite.

Algunas líneas relevantes del código fuente:

```python
x = self.input_embedding_table[(x,)]
x *= jnp.sqrt(self.embed_dim).astype(x.dtype)
```

## 3. Decoder

Cada _decoder_ se compone de una capa de _atención_ y otra de _feed forward_ con capas intermedias de normalización y conexión residual.

```
Input ─> Embedding ─> 26x(─> Norm ─> Atn ─> Norm ─> Add ─> Norm ─> FF ─> Norm ─> Add) ─> Output
                          │                          ↑  │                         ↑
                          └──────────────────────────┘  └─────────────────────────┘
```

Algunas líneas relevantes del código fuente:

```python
inputs_normalized = self.pre_attention_norm(x)

cache, attn_output = self.attn(
    inputs_normalized,
    segment_pos,
    cache,
    attn_mask,
)

attn_output = self.post_attention_norm(attn_output)

attn_output += x

outputs = self.pre_ffw_norm(attn_output)

outputs = self.mlp(outputs)

outputs = self.post_ffw_norm(outputs)

outputs += attn_output
```

### 3.1. Norm (Pre-Attention)

Esta capa normaliza la salida de la capa anterior con la función _RMSNorm_ (_Root Mean Square Layer Normalization_).

- [https://arxiv.org/pdf/1910.07467](https://arxiv.org/pdf/1910.07467)

La función ajusta los valores de entrada de acuerdo a la raíz cuadrada de la media de los cuadrados de los valores.

- $ \operatorname{RMS}(x) = \sqrt{\epsilon + \cfrac{1}{n} \sum\limits_{i=1}^{n} x_i^2} $

Los elementos de entrada se normalizan y multiplican por un factor de escala.

- $ y_i = \cfrac{x_i}{\operatorname{RMS}(x)} (1 + \gamma_i) $

La constante $\epsilon$ tiene valor $10^{-6}$. Y el vector de escalado $\gamma$ de la capa se inicializa con ceros y se actualiza durante el entrenamiento.

- $ \gamma = \begin{bmatrix} \gamma_1 & \gamma_2 & \cdots & \gamma_{1152} \end{bmatrix} $

Para el primer _decoder_, la salida de esta capa se calcula aplicando la función a su entrada, fila a fila.

- $ X \leftarrow \begin{bmatrix}
\operatorname{RMSNorm}(X_1)
\\ \operatorname{RMSNorm}(X_2)
\\ \vdots
\\ \operatorname{RMSNorm}(X_n)
\end{bmatrix} = \begin{bmatrix} 
\cfrac{x_{1,1}}{\operatorname{RMS}(X_1)} (1 + \gamma_1) & \cfrac{x_{1,2}}{\operatorname{RMS}(X_1)} (1 + \gamma_2) & \ldots & \cfrac{x_{1,1152}}{\operatorname{RMS}(X_1)} (1 + \gamma_{1152})
\\ \cfrac{x_{2,1}}{\operatorname{RMS}(X_2)} (1 + \gamma_1) & \cfrac{x_{2,2}}{\operatorname{RMS}(X_2)} (1 + \gamma_2) & \ldots & \cfrac{x_{2,1152}}{\operatorname{RMS}(X_2)} (1 + \gamma_{1152})
\\ \vdots & \vdots & \ddots & \vdots
\\ \cfrac{x_{n,1}}{\operatorname{RMS}(X_n)} (1 + \gamma_1) & \cfrac{x_{n,2}}{\operatorname{RMS}(X_n)} (1 + \gamma_2) & \ldots & \cfrac{x_{n,1152}}{\operatorname{RMS}(X_n)} (1 + \gamma_{1152})
\end{bmatrix} $

Para el resto de _decoders_, la salida se calcula aplicando la función a la salida del _decoder_ anterior.

Dimensiones:

- $ \gamma \in \mathbb{R}^{1 \times 1152} $

- $ X \in \mathbb{R}^{n \times 1152} $

Algunas líneas relevantes del código fuente:

```python
scale = self.param('scale', nn.initializers.zeros_init(), (x.shape[-1]))

var = jnp.mean(jnp.square(x), axis=-1, keepdims=True)
scale = jnp.expand_dims(scale, axis=range(len(x.shape) - 1))

normed_inputs = x * jax.lax.rsqrt(var + 1e-06)
normed_inputs = normed_inputs * (1 + scale)
```

### 3.2. Attention (Atn)

Esta capa aplica el mecanismo de atención.

```
Projection ─> QK-Norm ─> RoPE ─> Scale ─> Scores ─> Mask ─> SoftMax ─> Attention
```

Los modelos de la serie Gemma 3 implementan _Grouped Query Attention_ (_GQA_), pero el modelo 1B implementa _Multi-Query Attention_ (_MQA_).

- [https://arxiv.org/pdf/1911.02150](https://arxiv.org/pdf/1911.02150)

En la técnica más tradicional propuesta en el _paper_ original de la arquitectura _Transformer_ cada cabeza de atención tiene su propio conjunto de matrices $Q_i$, $K_i$ y $V_i$.

En _MQA_ cada cabeza de atención sólo tiene una matriz $Q_i$, y existen dos matrices $K$ y $V$ compartidas.

En _GQA_ cada cabeza de atención sólo tiene una matriz $Q_i$, y las cabezas se organizan en grupos, cada uno de ellos con dos matrices $K_j$ y $V_j$ compartidas por todas las cabezas del grupo.

#### 3.2.1. Projection

La capa tiene cuatro matrices de pesos distintas para _Query_, una para _Key_, y otra para _Value_. Todas ellas inicializadas con una distribución normal y actualizadas durante el entrenamiento.

- $ W_Q^1 \in \mathbb{R}^{1152 \times 256} $

- $ W_Q^2 \in \mathbb{R}^{1152 \times 256} $

- $ W_Q^3 \in \mathbb{R}^{1152 \times 256} $

- $ W_Q^4 \in \mathbb{R}^{1152 \times 256} $

- $ W_K \in \mathbb{R}^{1152 \times 256} $

- $ W_V \in \mathbb{R}^{1152 \times 256} $

Cada matriz proyecta las dimensiones de los elementos de entrada de 1.152 a 256, manteniendo la longitud $n$ de la secuencia.

- $ Q_1 = X W_Q^1 = \begin{bmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,1152}
\\ x_{2,1} & x_{2,2} & \ldots & x_{2,1152}
\\ \vdots & \vdots & \ddots & \vdots
\\ x_{n,1} & x_{n,2} & \ldots & x_{n,1152}
\end{bmatrix} \begin{bmatrix}
w^{q_1}_{1,1} & w^{q_1}_{1,2} & \ldots & w^{q_1}_{1,256}
\\ w^{q_1}_{2,1} & w^{q_1}_{2,2} & \ldots & w^{q_1}_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ w^{q_1}_{1152,1} & w^{q_1}_{1152,2} & \ldots & w^{q_1}_{1152,256}
\end{bmatrix} = \begin{bmatrix}
q^1_{1,1} & q^1_{1,2} & \ldots & q^1_{1,256}
\\ q^1_{2,1} & q^1_{2,2} & \ldots & q^1_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ q^1_{n,1} & q^1_{n,2} & \ldots & q^1_{n,256}
\end{bmatrix} $

- $ Q_2 = X W_Q^2 = \begin{bmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,1152}
\\ x_{2,1} & x_{2,2} & \ldots & x_{2,1152}
\\ \vdots & \vdots & \ddots & \vdots
\\ x_{n,1} & x_{n,2} & \ldots & x_{n,1152}
\end{bmatrix} \begin{bmatrix}
w^{q_2}_{1,1} & w^{q_2}_{1,2} & \ldots & w^{q_2}_{1,256}
\\ w^{q_2}_{2,1} & w^{q_2}_{2,2} & \ldots & w^{q_2}_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ w^{q_2}_{1152,1} & w^{q_2}_{1152,2} & \ldots & w^{q_2}_{1152,256}
\end{bmatrix} = \begin{bmatrix}
q^2_{1,1} & q^2_{1,2} & \ldots & q^2_{1,256}
\\ q^2_{2,1} & q^2_{2,2} & \ldots & q^2_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ q^2_{n,1} & q^2_{n,2} & \ldots & q^2_{n,256}
\end{bmatrix} $

- $ Q_3 = X W_Q^3 = \begin{bmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,1152}
\\ x_{2,1} & x_{2,2} & \ldots & x_{2,1152}
\\ \vdots & \vdots & \ddots & \vdots
\\ x_{n,1} & x_{n,2} & \ldots & x_{n,1152}
\end{bmatrix} \begin{bmatrix}
w^{q_3}_{1,1} & w^{q_3}_{1,2} & \ldots & w^{q_3}_{1,256}
\\ w^{q_3}_{2,1} & w^{q_3}_{2,2} & \ldots & w^{q_3}_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ w^{q_3}_{1152,1} & w^{q_3}_{1152,2} & \ldots & w^{q_3}_{1152,256}
\end{bmatrix} = \begin{bmatrix}
q^3_{1,1} & q^3_{1,2} & \ldots & q^3_{1,256}
\\ q^3_{2,1} & q^3_{2,2} & \ldots & q^3_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ q^3_{n,1} & q^3_{n,2} & \ldots & q^3_{n,256}
\end{bmatrix} $

- $ Q_4 = X W_Q^4 = \begin{bmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,1152}
\\ x_{2,1} & x_{2,2} & \ldots & x_{2,1152}
\\ \vdots & \vdots & \ddots & \vdots
\\ x_{n,1} & x_{n,2} & \ldots & x_{n,1152}
\end{bmatrix} \begin{bmatrix}
w^{q_4}_{1,1} & w^{q_4}_{1,2} & \ldots & w^{q_4}_{1,256}
\\ w^{q_4}_{2,1} & w^{q_4}_{2,2} & \ldots & w^{q_4}_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ w^{q_4}_{1152,1} & w^{q_4}_{1152,2} & \ldots & w^{q_4}_{1152,256}
\end{bmatrix} = \begin{bmatrix}
q^4_{1,1} & q^4_{1,2} & \ldots & q^4_{1,256}
\\ q^4_{2,1} & q^4_{2,2} & \ldots & q^4_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ q^4_{n,1} & q^4_{n,2} & \ldots & q^4_{n,256}
\end{bmatrix} $

- $ K = X W_K = \begin{bmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,1152}
\\ x_{2,1} & x_{2,2} & \ldots & x_{2,1152}
\\ \vdots & \vdots & \ddots & \vdots
\\ x_{n,1} & x_{n,2} & \ldots & x_{n,1152}
\end{bmatrix} \begin{bmatrix}
w^k_{1,1} & w^k_{1,2} & \ldots & w^k_{1,256}
\\ w^k_{2,1} & w^k_{2,2} & \ldots & w^k_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ w^k_{1152,1} & w^k_{1152,2} & \ldots & w^k_{1152,256}
\end{bmatrix} = \begin{bmatrix}
k_{1,1} & k_{1,2} & \ldots & k_{1,256}
\\ k_{2,1} & k_{2,2} & \ldots & k_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ k_{n,1} & k_{n,2} & \ldots & k_{n,256}
\end{bmatrix} $

- $ V = X W_V = \begin{bmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,1152}
\\ x_{2,1} & x_{2,2} & \ldots & x_{2,1152}
\\ \vdots & \vdots & \ddots & \vdots
\\ x_{n,1} & x_{n,2} & \ldots & x_{n,1152}
\end{bmatrix} \begin{bmatrix}
w^v_{1,1} & w^v_{1,2} & \ldots & w^v_{1,256}
\\ w^v_{2,1} & w^v_{2,2} & \ldots & w^v_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ w^v_{1152,1} & w^v_{1152,2} & \ldots & w^v_{1152,256}
\end{bmatrix} = \begin{bmatrix}
v_{1,1} & v_{1,2} & \ldots & v_{1,256}
\\ v_{2,1} & v_{2,2} & \ldots & v_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ v_{n,1} & v_{n,2} & \ldots & v_{n,256}
\end{bmatrix} $

Dimensiones:

- $ Q_1 \in \mathbb{R}^{n \times 256} $

- $ Q_2 \in \mathbb{R}^{n \times 256} $

- $ Q_3 \in \mathbb{R}^{n \times 256} $

- $ Q_4 \in \mathbb{R}^{n \times 256} $

- $ K \in \mathbb{R}^{n \times 256} $

- $ V \in \mathbb{R}^{n \times 256} $

Para evitar repetir cálculos, y reducir la memoria utilizada, se utiliza una _cache_ sobre las matrices $K$ y $V$ de la capa de atención de cada _decoder_.

Huelga decir que el modelo gestiona eficientemente los cálculos con matrices, pero por simplicidad en el análisis se muestra una capa con sus matrices claramente diferenciadas sobre las que se aplican operaciones de forma individual paso a paso.

Algunas líneas relevantes del código fuente:

```python
self.q_einsum = _layers.Einsum(
  shape=(self.num_heads, self.features, self.head_dim),
)
self.kv_einsum = _layers.Einsum(
  shape=(2, self.num_kv_heads, self.features, self.head_dim),
)

query_proj = self.q_einsum('BTD,NDH->BTNH', x)
key_proj, value_proj = self.kv_einsum('BSD,CKDH->CBSKH', x)
```

#### 3.2.2. QK-Norm

Las proyecciones $Q_i$ y $K$ se normalizan aplicando _RMSNorm_ fila a fila.

- $ Q_1 \leftarrow \begin{bmatrix}
\operatorname{RMSNorm}(Q^1_1)
\\ \operatorname{RMSNorm}(Q^1_2)
\\ \vdots
\\ \operatorname{RMSNorm}(Q^1_n)
\end{bmatrix} \quad Q_2 \leftarrow \begin{bmatrix}
\operatorname{RMSNorm}(Q^2_1)
\\ \operatorname{RMSNorm}(Q^2_2)
\\ \vdots
\\ \operatorname{RMSNorm}(Q^2_n)
\end{bmatrix} \quad Q_3 \leftarrow \begin{bmatrix}
\operatorname{RMSNorm}(Q^3_1)
\\ \operatorname{RMSNorm}(Q^3_2)
\\ \vdots
\\ \operatorname{RMSNorm}(Q^3_n)
\end{bmatrix} \quad Q_4 \leftarrow \begin{bmatrix}
\operatorname{RMSNorm}(Q^4_1)
\\ \operatorname{RMSNorm}(Q^4_2)
\\ \vdots
\\ \operatorname{RMSNorm}(Q^4_n)
\end{bmatrix} $

- $ K \leftarrow \begin{bmatrix}
\operatorname{RMSNorm}(K_1)
\\ \operatorname{RMSNorm}(K_2)
\\ \vdots
\\ \operatorname{RMSNorm}(K_n)
\end{bmatrix} $

La constante $\epsilon$ tiene valor $10^{-6}$. Y los vectores de escalado $\gamma^Q$ y $\gamma^K$ de la capa se inicializan con ceros y se actualizan durante el entrenamiento.

Dimensiones:

- $ \gamma^Q \in \mathbb{R}^{1 \times 256} $

- $ \gamma^K \in \mathbb{R}^{1 \times 256} $

Las dimensiones de las matrices de proyección no se ven afectadas.

Algunas líneas relevantes del código fuente:

```python
query_proj = self._query_norm(query_proj)
key_proj = self._key_norm(key_proj)
```

#### 3.2.3. 5-to-1 Interleaved Attention

El modelo intercala capas de atención local con capas de atención global.

Cada 5 capas de atención local intercala 1 capa de atención global.

El modelo tiene una ventana de contexto de 32.768 (32K) _tokens_.

Las capas de atención local tienen una ventana de contexto de 512 _tokens_.

Algunas líneas relevantes del código fuente:

```python
GEMMA3_ATTENTION_PATTERN = (
    _modules.AttentionType.LOCAL_SLIDING,
    _modules.AttentionType.LOCAL_SLIDING,
    _modules.AttentionType.LOCAL_SLIDING,
    _modules.AttentionType.LOCAL_SLIDING,
    _modules.AttentionType.LOCAL_SLIDING,
    _modules.AttentionType.GLOBAL,
)
```

#### 3.2.4. RoPE

Las proyecciones $Q_i$ y $K$ se enriquecen con información posicional mediante _Rotary Positional Embedding_ (_RoPE_).

- [https://arxiv.org/pdf/2104.09864](https://arxiv.org/pdf/2104.09864)

Esta técnica es la más usada actualmente, en contraposición a la técnica _Positional Encoding_ propuesta en el _paper_ original de la arquitectura _Transformer_.

- $ Q_1 \leftarrow \operatorname{RoPE}(Q_1)
\quad Q_2 \leftarrow \operatorname{RoPE}(Q_2)
\quad Q_3 \leftarrow \operatorname{RoPE}(Q_3)
\quad Q_4 \leftarrow \operatorname{RoPE}(Q_4) $

- $ K \leftarrow \operatorname{RoPE}(K) $

Se aplica una función $RoPE(m, j)$ a las matrices $Q_i$ y $K$ que se evalúa para cada par $(m, j)$. Donde $m$ es una posición dentro de la secuencia y $j$ es una pareja de elementos de las matrices.

Es decir, se recorren todas las filas para $m = 1, 2, ..., n$, y se toman los elementos de cada fila de dos en dos $(x_{2j}, x_{2j+1})$ para $j = 0, 1, \ldots, \cfrac{256}{2} - 1$ (por convención se empieza en $0$).

Para cada par $(m, j)$ se calcula un ángulo $\theta_{m,j}$, y se rota cada pareja de elementos por dicho ángulo como si fueran coordenadas 2D.

- $ \theta_{m,j} = \cfrac{m}{\text{baseFrequency}^{\left(\cfrac{2j}{256}\right)}} $

- $ x'_{2j} = x_{2j} \cos(\theta_{m,j}) - x_{2j+1} \sin(\theta_{m,j}) $

- $ x'_{2j+1} = x_{2j} \sin(\theta_{m,j}) + x_{2j+1} \cos(\theta_{m,j}) $

Las capas de atención local utilizan un frecuencia base de 10K, y las de atención global de 1M.

Las dimensiones de las matrices de proyección no se ven afectadas.

Algunas líneas relevantes del código fuente:

```python
query_proj = _positional_embeddings.apply_rope(
  query_proj,
  segment_pos,
  base_frequency=self.rope_base_frequency,
  scale_factor=self.rope_scale_factor,
)

key_proj = _positional_embeddings.apply_rope(
  key_proj,
  segment_pos,
  base_frequency=self.rope_base_frequency,
  scale_factor=self.rope_scale_factor,
)
```

#### 3.2.5. Scale

Las proyecciones $Q_i$ se escalan por la inversa de la raíz cuadrada de las dimensiones de las cabezas de atención siguiendo el _paper_ original de la arquitectura _Transformer_.

- $ Q_1 \leftarrow Q_1 \cfrac{1}{\sqrt{256}}
\quad Q_2 \leftarrow Q_2 \cfrac{1}{\sqrt{256}}
\quad Q_3 \leftarrow Q_3 \cfrac{1}{\sqrt{256}}
\quad Q_4 \leftarrow Q_4 \cfrac{1}{\sqrt{256}} $

Las dimensiones de las matrices de proyección no se ven afectadas.

Algunas líneas relevantes del código fuente:

```python
query_scaled = query_proj * self.query_pre_attn_scalar
```

#### 3.2.6. Scores

La matrices de "_scores_" se calculan multiplicando las proyecciones $Q_i$ por la traspuesta de $K$.

- $ S_1 = Q_1 K^T = \begin{bmatrix}
q^1_{1,1} & q^1_{1,2} & \ldots & q^1_{1,256}
\\ q^1_{2,1} & q^1_{2,2} & \ldots & q^1_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ q^1_{n,1} & q^1_{n,2} & \ldots & q^1_{n,256}
\end{bmatrix} \begin{bmatrix}
k_{1,1} & k_{1,2} & \ldots & k_{1,n}
\\ k_{2,1} & k_{2,2} & \ldots & k_{2,n}
\\ \vdots & \vdots & \ddots & \vdots
\\ k_{256,1} & k_{256,2} & \ldots & k_{256,n}
\end{bmatrix} = \begin{bmatrix}
s^1_{1,1} & s^1_{1,2} & \ldots & s^1_{1,n}
\\ s^1_{2,1} & s^1_{2,2} & \ldots & s^1_{2,n}
\\ \vdots & \vdots & \ddots & \vdots
\\ s^1_{n,1} & s^1_{n,2} & \ldots & s^1_{n,n}
\end{bmatrix} $

- $ S_2 = Q_2 K^T = \begin{bmatrix}
q^2_{1,1} & q^2_{1,2} & \ldots & q^2_{1,256}
\\ q^2_{2,1} & q^2_{2,2} & \ldots & q^2_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ q^2_{n,1} & q^2_{n,2} & \ldots & q^2_{n,256}
\end{bmatrix} \begin{bmatrix}
k_{1,1} & k_{1,2} & \ldots & k_{1,n}
\\ k_{2,1} & k_{2,2} & \ldots & k_{2,n}
\\ \vdots & \vdots & \ddots & \vdots
\\ k_{256,1} & k_{256,2} & \ldots & k_{256,n}
\end{bmatrix} = \begin{bmatrix}
s^2_{1,1} & s^2_{1,2} & \ldots & s^2_{1,n}
\\ s^2_{2,1} & s^2_{2,2} & \ldots & s^2_{2,n}
\\ \vdots & \vdots & \ddots & \vdots
\\ s^2_{n,1} & s^2_{n,2} & \ldots & s^2_{n,n}
\end{bmatrix} $

- $ S_3 = Q_3 K^T = \begin{bmatrix}
q^3_{1,1} & q^3_{1,2} & \ldots & q^3_{1,256}
\\ q^3_{2,1} & q^3_{2,2} & \ldots & q^3_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ q^3_{n,1} & q^3_{n,2} & \ldots & q^3_{n,256}
\end{bmatrix} \begin{bmatrix}
k_{1,1} & k_{1,2} & \ldots & k_{1,n}
\\ k_{2,1} & k_{2,2} & \ldots & k_{2,n}
\\ \vdots & \vdots & \ddots & \vdots
\\ k_{256,1} & k_{256,2} & \ldots & k_{256,n}
\end{bmatrix} = \begin{bmatrix}
s^3_{1,1} & s^3_{1,2} & \ldots & s^3_{1,n}
\\ s^3_{2,1} & s^3_{2,2} & \ldots & s^3_{2,n}
\\ \vdots & \vdots & \ddots & \vdots
\\ s^3_{n,1} & s^3_{n,2} & \ldots & s^3_{n,n}
\end{bmatrix} $

- $ S_4 = Q_4 K^T = \begin{bmatrix}
q^4_{1,1} & q^4_{1,2} & \ldots & q^4_{1,256}
\\ q^4_{2,1} & q^4_{2,2} & \ldots & q^4_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ q^4_{n,1} & q^4_{n,2} & \ldots & q^4_{n,256}
\end{bmatrix} \begin{bmatrix}
k_{1,1} & k_{1,2} & \ldots & k_{1,n}
\\ k_{2,1} & k_{2,2} & \ldots & k_{2,n}
\\ \vdots & \vdots & \ddots & \vdots
\\ k_{256,1} & k_{256,2} & \ldots & k_{256,n}
\end{bmatrix} = \begin{bmatrix}
s^4_{1,1} & s^4_{1,2} & \ldots & s^4_{1,n}
\\ s^4_{2,1} & s^4_{2,2} & \ldots & s^4_{2,n}
\\ \vdots & \vdots & \ddots & \vdots
\\ s^4_{n,1} & s^4_{n,2} & \ldots & s^4_{n,n}
\end{bmatrix}$

Esta operación crea matrices cuadradas de $n \times n$, donde cada elemento representa la atención que cada _token_ presta a si mismo y al resto de _tokens_ de la secuencia.

Dimensiones:

- $ Q_1 \in \mathbb{R}^{n \times 256} $

- $ Q_2 \in \mathbb{R}^{n \times 256} $

- $ Q_3 \in \mathbb{R}^{n \times 256} $

- $ Q_4 \in \mathbb{R}^{n \times 256} $

- $ K \in \mathbb{R}^{n \times 256} $

- $ S_1 \in \mathbb{R}^{n \times n} $

- $ S_2 \in \mathbb{R}^{n \times n} $

- $ S_3 \in \mathbb{R}^{n \times n} $

- $ S_4 \in \mathbb{R}^{n \times n} $

Algunas líneas relevantes del código fuente:

```python
logits = jnp.einsum('BTNH,BSNH->BTNS', query_scaled, key_proj)
```

#### 3.2.7. Mask

Aunque se ha omitido por simplicidad en el análisis, el modelo trabaja con _batches_. Es decir, procesa varias secuencias de entrada a un mismo tiempo.

Como cada secuencia puede tener una longitud distinta, se utilizan _tokens_ de _padding_ para completar la secuencias más cortas, y que todas tengan la misma longitud.

Los _tokens_ de _padding_ se ignoran durante el procesamiento mediante máscaras. Siendo las máscaras matrices numéricas o lógicas que indican que elementos se deben ignorar.

De igual forma, un _token_ $i$ no puede prestar atención a un _token_ $j$ que aún no se ha generado. Por lo que se aplican máscaras sobre las matrices de _scores_.

Además, para las capas de atención local se construyen máscaras de atención local sobre el tamaño de la ventana de contexto local, que en este modelo es de 512 _tokens_.

Las máscaras se combinan y aplican sobre la matriz de _scores_, sustituyendo los valores a ignorar por un número negativo muy pequeño. En el _paper_ original de la arquitectura _Transformers_ se propone utilizar $-\infty$, pero el modelo utiliza una constante con valor $-2.3819763e38$.

- $ S_1 \leftarrow \operatorname{mask}(S_1)
\quad S_2 \leftarrow \operatorname{mask}(S_2)
\quad S_3 \leftarrow \operatorname{mask}(S_3)
\quad S_4 \leftarrow \operatorname{mask}(S_4) $

Las dimensiones de las matrices de _scores_ no se ven afectadas.

Algunas líneas relevantes del código fuente:

```python
if self.attn_type == AttentionType.LOCAL_SLIDING:
  sliding_mask = _create_sliding_mask(
    segment_pos,
    end_index=cache['end_index'][0] if cache is not None else 0,
    cache_len=attn_mask.shape[-1],
    sliding_window_size=self.sliding_window_size,
  )
  attn_mask *= sliding_mask

padded_logits = jnp.where((jnp.expand_dims(attn_mask, -2)), logits, K_MASK)
```

#### 3.2.8. SoftMax

Los valores de las matrices de _scores_ se convierten a una distribución de probabilidad aplicando la función $softmax$ fila a fila.

- $ S_1 \leftarrow \begin{bmatrix}
\operatorname{softmax}(S^1_1)
\\ \operatorname{softmax}(S^1_2)
\\ \vdots
\\ \operatorname{softmax}(S^1_n)
\end{bmatrix}
\quad S_2 \leftarrow \begin{bmatrix}
\operatorname{softmax}(S^2_1)
\\ \operatorname{softmax}(S^2_2)
\\ \vdots
\\ \operatorname{softmax}(S^2_n)
\end{bmatrix}
\quad S_3 \leftarrow \begin{bmatrix}
\operatorname{softmax}(S^3_1)
\\ \operatorname{softmax}(S^3_2)
\\ \vdots
\\ \operatorname{softmax}(S^3_n)
\end{bmatrix}
\quad S_4 \leftarrow \begin{bmatrix}
\operatorname{softmax}(S^4_1)
\\ \operatorname{softmax}(S^4_2)
\\ \vdots
\\ \operatorname{softmax}(S^4_n)
\end{bmatrix} $

La función $softmax$ comprime valores que se encuentra dentro de un rango arbitrario al rango $[0, 1]$, de forma que la suma de cada fila sea igual a $1$.

- $ \operatorname{softmax}(x)_i = \cfrac{e^{x_i}}{\sum\limits_{j=1}^{d} e^{x_j}} $

Debe ser claro que al aplicar la máscara de atención y sustituir los valores de los elementos de la matriz no deseados por $-\infty$ (o un número negativo muy pequeño) tiene el efecto de que se ignoren, ya que $e^{-\infty} = 0$.

Las dimensiones de las matrices de _scores_ no se ven afectadas.

Algunas líneas relevantes del código fuente:

```python
probs = jax.nn.softmax(padded_logits, axis=-1).astype(key_proj.dtype)
```

#### 3.2.9. Attention

Las matrices de _scores_ se proyectan por la matriz de _Value_ para obtener las matrices de atención.

- $ A_1 = S_1 V = \begin{bmatrix}
s^1_{1,1} & s^1_{1,2} & \ldots & s^1_{1,n}
\\ s^1_{2,1} & s^1_{2,2} & \ldots & s^1_{2,n}
\\ \vdots & \vdots & \ddots & \vdots
\\ s^1_{n,1} & s^1_{n,2} & \ldots & s^1_{n,n}
\end{bmatrix} \begin{bmatrix}
v_{1,1} & v_{1,2} & \ldots & v_{1,256}
\\ v_{2,1} & v_{2,2} & \ldots & v_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ v_{n,1} & v_{n,2} & \ldots & v_{n,256}
\end{bmatrix} = \begin{bmatrix}
a^1_{1,1} & a^1_{1,2} & \ldots & a^1_{1,256}
\\ a^1_{2,1} & a^1_{2,2} & \ldots & a^1_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ a^1_{n,1} & a^1_{n,2} & \ldots & a^1_{n,256}
\end{bmatrix} $

- $ A_2 = S_2 V = \begin{bmatrix}
s^2_{1,1} & s^2_{1,2} & \ldots & s^2_{1,n}
\\ s^2_{2,1} & s^2_{2,2} & \ldots & s^2_{2,n}
\\ \vdots & \vdots & \ddots & \vdots
\\ s^2_{n,1} & s^2_{n,2} & \ldots & s^2_{n,n}
\end{bmatrix} \begin{bmatrix}
v_{1,1} & v_{1,2} & \ldots & v_{1,256}
\\ v_{2,1} & v_{2,2} & \ldots & v_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ v_{n,1} & v_{n,2} & \ldots & v_{n,256}
\end{bmatrix} = \begin{bmatrix}
a^2_{1,1} & a^2_{1,2} & \ldots & a^2_{1,256}
\\ a^2_{2,1} & a^2_{2,2} & \ldots & a^2_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ a^2_{n,1} & a^2_{n,2} & \ldots & a^2_{n,256}
\end{bmatrix} $

- $ A_3 = S_3 V = \begin{bmatrix}
s^3_{1,1} & s^3_{1,2} & \ldots & s^3_{1,n}
\\ s^3_{2,1} & s^3_{2,2} & \ldots & s^3_{2,n}
\\ \vdots & \vdots & \ddots & \vdots
\\ s^3_{n,1} & s^3_{n,2} & \ldots & s^3_{n,n}
\end{bmatrix} \begin{bmatrix}
v_{1,1} & v_{1,2} & \ldots & v_{1,256}
\\ v_{2,1} & v_{2,2} & \ldots & v_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ v_{n,1} & v_{n,2} & \ldots & v_{n,256}
\end{bmatrix} = \begin{bmatrix}
a^3_{1,1} & a^3_{1,2} & \ldots & a^3_{1,256}
\\ a^3_{2,1} & a^3_{2,2} & \ldots & a^3_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ a^3_{n,1} & a^3_{n,2} & \ldots & a^3_{n,256}
\end{bmatrix} $

- $ A_4 = S_4 V = \begin{bmatrix}
s^4_{1,1} & s^4_{1,2} & \ldots & s^4_{1,n}
\\ s^4_{2,1} & s^4_{2,2} & \ldots & s^4_{2,n}
\\ \vdots & \vdots & \ddots & \vdots
\\ s^4_{n,1} & s^4_{n,2} & \ldots & s^4_{n,n}
\end{bmatrix} \begin{bmatrix}
v_{1,1} & v_{1,2} & \ldots & v_{1,256}
\\ v_{2,1} & v_{2,2} & \ldots & v_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ v_{n,1} & v_{n,2} & \ldots & v_{n,256}
\end{bmatrix} = \begin{bmatrix}
a^4_{1,1} & a^4_{1,2} & \ldots & a^4_{1,256}
\\ a^4_{2,1} & a^4_{2,2} & \ldots & a^4_{2,256}
\\ \vdots & \vdots & \ddots & \vdots
\\ a^4_{n,1} & a^4_{n,2} & \ldots & a^4_{n,256}
\end{bmatrix} $

Las matrices de atención se concatenan para formar una única matriz.

- $ A = \operatorname{concat}(A_1, A_2, A_3, A_4) $

- $ A = \begin{bmatrix}
a^1_{1,1} & a^1_{1,2} & \ldots & a^1_{1,256} & a^2_{1,1} & a^2_{1,2} & \ldots & a^2_{1,256} & a^3_{1,1} & a^3_{1,2} & \ldots & a^3_{1,256} & a^4_{1,1} & a^4_{1,2} & \ldots & a^4_{1,256}
\\ a^1_{2,1} & a^1_{2,2} & \ldots & a^1_{2,256} & a^2_{2,1} & a^2_{2,2} & \ldots & a^2_{2,256} & a^3_{2,1} & a^3_{2,2} & \ldots & a^3_{2,256} & a^4_{2,1} & a^4_{2,2} & \ldots & a^4_{2,256}
\\ \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \ddots & \vdots
\\ a^1_{n,1} & a^1_{n,2} & \ldots & a^1_{n,256} & a^2_{n,1} & a^2_{n,2} & \ldots & a^2_{n,256} & a^3_{n,1} & a^3_{n,2} & \ldots & a^3_{n,256} & a^4_{n,1} & a^4_{n,2} & \ldots & a^4_{n,256}
\end{bmatrix} = \begin{bmatrix}
a_{1,1} & a_{1,2} & \ldots & a_{1,1024}
\\ a_{2,1} & a_{2,2} & \ldots & a_{2,1024}
\\ \vdots & \vdots & \ddots & \vdots
\\ a_{n,1} & a_{n,2} & \ldots & a_{n,1024}
\end{bmatrix} $

Y finalmente, la salida de capa de atención se calcula proyectando $A$ por una nueva matriz $W_O$, inicializada con una distribución normal y actualizada durante el entrenamiento, que retorna la atención a las dimensiones del modelo.

- $ X \leftarrow A W_O = \begin{bmatrix}
a_{1,1} & a_{1,2} & \ldots & a_{1,1024}
\\ a_{2,1} & a_{2,2} & \ldots & a_{2,1024}
\\ \vdots & \vdots & \ddots & \vdots
\\ a_{n,1} & a_{n,2} & \ldots & a_{n,1024}
\end{bmatrix} \begin{bmatrix}
w^o_{1,1} & w^o_{1,2} & \ldots & w^o_{1,1152}
\\ w^o_{2,1} & w^o_{2,2} & \ldots & w^o_{2,1152}
\\ \vdots & \vdots & \ddots & \vdots
\\ w^o_{1024,1} & w^o_{1024,2} & \ldots & w^o_{1024,1152}
\end{bmatrix} $

Dimensiones:

- $ S_1 \in \mathbb{R}^{n \times n} $

- $ S_2 \in \mathbb{R}^{n \times n} $

- $ S_3 \in \mathbb{R}^{n \times n} $

- $ S_4 \in \mathbb{R}^{n \times n} $

- $ V \in \mathbb{R}^{n \times 256} $

- $ A_i \in \mathbb{R}^{n \times 256} $

- $ A \in \mathbb{R}^{n \times 1024} $

- $ W_o \in \mathbb{R}^{1024 \times 1152} $

- $ X \in \mathbb{R}^{n \times 1152} $

Algunas líneas relevantes del código fuente:

```python
encoded = jnp.einsum('BTNS,BSNH->BTNH', probs, value_proj)

attn_output = self.attn_vec_einsum('BTNH,NHD->BTD', encoded)
```

### 3.3. Norm (Post-Attention)

Esta capa normaliza la salida de la capa anterior con _RMSNorm_.

La salida de esta capa se calcula aplicando la función a su entrada fila a fila.

- $ X \leftarrow \begin{bmatrix}
\operatorname{RMSNorm}(X_1)
\\ \operatorname{RMSNorm}(X_2)
\\ \vdots
\\ \operatorname{RMSNorm}(X_n)
\end{bmatrix} $

La constante $\epsilon$ tiene valor $10^{-6}$. Y el vector de escalado $\gamma$ de la capa se inicializa con ceros y se actualiza durante el entrenamiento.

Dimensiones:

- $ \gamma \in \mathbb{R}^{1 \times 1152} $

- $ X \in \mathbb{R}^{n \times 1152} $

### 3.4. Add (Post-Attention)

Esta capa realiza una conexión residual para propagar información presente antes de la atención.

Para el primer _decoder_, la salida de esta capa se calcula sumando a la salida de la capa anterior la salida de la capa inicial de _embeddings_.

- $ \text{ATN} = X + \text{EMB} $

Y para el resto de _decoders_, la salida se calcula sumando a la salida de la capa anterior la salida del _decoder_ anterior.

La salida de este paso es el resultado de la suma.

- $ X \leftarrow \text{ATN} $

Dimensiones:

- $ X \in \mathbb{R}^{n \times 1152} $

- $ \text{EMB} \in \mathbb{R}^{n \times 1152} $

- $ \text{ATN} \in \mathbb{R}^{n \times 1152} $

### 3.5. Norm (Pre-Feed Forward)

Esta capa normaliza la salida de la capa anterior con _RMSNorm_.

La salida de esta capa se calcula aplicando la función a su entrada fila a fila.

- $ X \leftarrow \begin{bmatrix}
\operatorname{RMSNorm}(X_1)
\\ \operatorname{RMSNorm}(X_2)
\\ \vdots
\\ \operatorname{RMSNorm}(X_n)
\end{bmatrix} $

La constante $\epsilon$ tiene valor $10^{-6}$. Y el vector de escalado $\gamma$ de la capa se inicializa con ceros y se actualiza durante el entrenamiento.

Dimensiones:

- $ \gamma \in \mathbb{R}^{1 \times 1152} $

- $ X \in \mathbb{R}^{n \times 1152} $

### 3.6. Feed Forward (FF)

Esta capa añade no linealidad expandiendo las dimensiones del modelo para mejorar la potencia expresiva del mismo.

Utiliza _GeGLU_, una variante de _GLU_ (_Gated Linear Unit_) con la función de activación _GELU_.

- [https://arxiv.org/pdf/2002.05202](https://arxiv.org/pdf/2002.05202)

Aplica dos proyecciones lineales a partir de los datos de entrada para expandir las dimensiones originales de los _embeddings_, de 1.152 a 6.912 (seis veces el tamaño original).

- $ g_1 = x W_1 $

- $ g_2 = x W_2 $

Aplica la función _GELU_ a la primera proyección y multiplica el resultado, elemento a elemento, por la segunda proyección.

- $ h = \operatorname{GELU}(g_1) \otimes g_2 $

Y, finalmente, reduce las dimensiones a las originales, de 6.912 a 1.152.

- $ y = h W_3 $

La segunda proyección funciona como una puerta de control (_gate_), que deja pasar una cantidad mayor o menor de información del resultado de aplicar la función de activación a la primera proyección.

En total hay tres matrices de pesos que son inicializadas con una distribución normal y actualizadas durante el entrenamiento.

La salida de esta capa se calcula aplicando _GeGLU_ y las proyecciones a su entrada.

- $ g_1 = X W_1 = \begin{bmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,1152}
\\ x_{2,1} & x_{2,2} & \ldots & x_{2,1152}
\\ \vdots & \vdots & \ddots & \vdots
\\ x_{n,1} & x_{n,2} & \ldots & x_{n,1152}
\end{bmatrix} \begin{bmatrix}
w^1_{1,1} & w^1_{1,2} & \ldots & w^1_{1,6912}
\\ w^1_{2,1} & w^1_{2,2} & \ldots & w^1_{2,6912}
\\ \vdots & \vdots & \ddots & \vdots
\\ w^1_{1152,1} & w^1_{1152,2} & \ldots & w^1_{1152,6912}
\end{bmatrix} = \begin{bmatrix}
g^1_{1,1} & g^1_{1,2} & \ldots & g^1_{1,6912}
\\ g^1_{2,1} & g^1_{2,2} & \ldots & g^1_{2,6912}
\\ \vdots & \vdots & \ddots & \vdots
\\ g^1_{n,1} & g^1_{n,2} & \ldots & g^1_{n,6912}
\end{bmatrix} $

- $ g_2 = X W_2 = \begin{bmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,1152}
\\ x_{2,1} & x_{2,2} & \ldots & x_{2,1152}
\\ \vdots & \vdots & \ddots & \vdots
\\ x_{n,1} & x_{n,2} & \ldots & x_{n,1152}
\end{bmatrix} \begin{bmatrix}
w^2_{1,1} & w^2_{1,2} & \ldots & w^2_{1,6912}
\\ w^2_{2,1} & w^2_{2,2} & \ldots & w^2_{2,6912}
\\ \vdots & \vdots & \ddots & \vdots
\\ w^2_{1152,1} & w^2_{1152,2} & \ldots & w^2_{1152,6912}
\end{bmatrix} = \begin{bmatrix}
g^2_{1,1} & g^2_{1,2} & \ldots & g^2_{1,6912}
\\ g^2_{2,1} & g^2_{2,2} & \ldots & g^2_{2,6912}
\\ \vdots & \vdots & \ddots & \vdots
\\ g^2_{n,1} & g^2_{n,2} & \ldots & g^2_{n,6912}
\end{bmatrix} $

- $ h = \operatorname{GELU}(g_1) \otimes g_2 = \begin{bmatrix}
\operatorname{GELU}(g^1_{1,1}) g^2_{1,1} & \operatorname{GELU}(g^1_{1,2}) g^2_{1,2} & \ldots & \operatorname{GELU}(g^1_{1,6912}) g^2_{1,6912}
\\ \operatorname{GELU}(g^1_{2,1}) g^2_{2,1} & \operatorname{GELU}(g^1_{2,2}) g^2_{2,2} & \ldots & \operatorname{GELU}(g^1_{2,6912}) g^2_{2,6912}
\\ \vdots & \vdots & \ddots & \vdots
\\ \operatorname{GELU}(g^1_{n,1}) g^2_{n,1} & \operatorname{GELU}(g^1_{n,2}) g^2_{n,2} & \ldots & \operatorname{GELU}(g^1_{n,6912}) g^2_{n,6912}
\end{bmatrix} = \begin{bmatrix}
h_{1,1} & h_{1,2} & \ldots & h_{1,6912}
\\ h_{2,1} & h_{2,2} & \ldots & h_{2,6912}
\\ \vdots & \vdots & \ddots & \vdots
\\ h_{n,1} & h_{n,2} & \ldots & h_{n,6912}
\end{bmatrix} $

- $ X \leftarrow h W_3 = \begin{bmatrix}
h_{1,1} & h_{1,2} & \ldots & h_{1,6912}
\\ h_{2,1} & h_{2,2} & \ldots & h_{2,6912}
\\ \vdots & \vdots & \ddots & \vdots
\\ h_{n,1} & h_{n,2} & \ldots & h_{n,6912}
\end{bmatrix} \begin{bmatrix}
w^3_{1,1} & w^3_{1,2} & \ldots & w^3_{1,1152}
\\ w^3_{2,1} & w^3_{2,2} & \ldots & w^3_{2,1152}
\\ \vdots & \vdots & \ddots & \vdots
\\ w^3_{6912,1} & w^3_{6912,2} & \ldots & w^3_{6912,1152}
\end{bmatrix} $

Dimensiones:

- $ W_1 \in \mathbb{R}^{1152 \times 6912} $

- $ g_1 \in \mathbb{R}^{n \times 6912} $

- $ W_2 \in \mathbb{R}^{1152 \times 6912} $

- $ g_2 \in \mathbb{R}^{n \times 6912} $

- $ h \in \mathbb{R}^{n \times 6912} $

- $ W_3 \in \mathbb{R}^{6912 \times 1152} $

- $ X \in \mathbb{R}^{n \times 1152} $

Algunas líneas relevantes del código fuente:

```python
gating = _layers.Einsum(
  shape=(2, self.features, self.hidden_dim),
  weight_name='gating_einsum',
)

gate = gating(eq, x)

activations = nn.gelu(gate[..., 0, :]) * gate[..., 1, :]

linear = _layers.Einsum(
  shape=(self.hidden_dim, self.features),
  weight_name='linear',
)

outputs = linear('...H,HF->...F', activations)
```

### 3.7. Norm (Post-Feed Forward)

Esta capa normaliza la salida de la capa anterior con _RMSNorm_.

La salida de esta capa se calcula aplicando la función a su entrada fila a fila.

- $ X \leftarrow \begin{bmatrix}
\operatorname{RMSNorm}(X_1)
\\ \operatorname{RMSNorm}(X_2)
\\ \vdots
\\ \operatorname{RMSNorm}(X_n)
\end{bmatrix} $

La constante $\epsilon$ tiene valor $10^{-6}$. Y el vector de escalado $\gamma$ de la capa se inicializa con ceros y se actualiza durante el entrenamiento.

Dimensiones:

- $ \gamma \in \mathbb{R}^{1 \times 1152} $

- $ X \in \mathbb{R}^{n \times 1152} $

### 3.8. Add (Post-Feed Forward)

Esta capa realiza una conexión residual para propagar información presente antes de añadir la no linealidad.

La salida de esta capa se calcula sumando a la salida de la capa anterior la salida de la atención.

- $ X \leftarrow X + \text{ATN} $

Dimensiones:

- $ X \in \mathbb{R}^{n \times 1152} $

- $ \text{ATN} \in \mathbb{R}^{n \times 1152} $

## 4. Output

En la salida del modelo se aplica una normalización final y se obtienen las probabilidades (no normalizadas) para cada _token_ del vocabulario.

```
Input ─> Embedding ─> 26 x Decoder ─> Norm ─> Logits -> Output
```

Algunas líneas relevantes del código fuente:

```python
x = self.final_norm(x)

logits = self.embedder.decode(x)
```

### 4.1. Norm (Final)

Esta capa normaliza la salida de la capa anterior con _RMSNorm_.

La salida de esta capa se calcula aplicando la función a la salida del último _decoder_ fila a fila.

- $ X \leftarrow \begin{bmatrix}
\operatorname{RMSNorm}(X_1)
\\ \operatorname{RMSNorm}(X_2)
\\ \vdots
\\ \operatorname{RMSNorm}(X_n)
\end{bmatrix} $

La constante $\epsilon$ tiene valor $10^{-6}$. Y el vector de escalado $\gamma$ de la capa se inicializa con ceros y se actualiza durante el entrenamiento.

Dimensiones:

- $ \gamma \in \mathbb{R}^{1 \times 1152} $

- $ X \in \mathbb{R}^{n \times 1152} $

### 4.2. Logits

Finalmente, la salida del modelo son los conocidos como "_logits_".

Es decir, las puntuaciones brutas (_scores_) para cada _token_ del vocabulario que indican la probabilidad (no normalizada) que tienen de ser el siguiente de la secuencia de salida.

La capa proyecta las dimensiones internas del modelo a las dimensiones del vocabulario, de 1.152 a 256.128, a través de la traspuesta de la matriz de pesos de la capa de _embedding_ utilizada a la entrada del modelo.

La salida de esta capa se calcula multiplicando su entrada por la traspuesta de la matriz de pesos.

- $ Y = X W^T = \begin{bmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,1152}
\\ x_{2,1} & x_{2,2} & \ldots & x_{2,1152}
\\ \vdots & \vdots & \ddots & \vdots
\\ x_{n,1} & x_{n,2} & \ldots & x_{n,1152}
\end{bmatrix} \begin{bmatrix}
w_{1,1} & w_{1,2} & \ldots & w_{1,256128}
\\ w_{2,1} & w_{2,2} & \ldots & w_{2,256128}
\\ \vdots & \vdots & \ddots & \vdots
\\ w_{1152,1} & w_{1152,2} & \ldots & w_{1152,256128}
\end{bmatrix} = \begin{bmatrix}
y_{1,1} & y_{1,2} & \ldots & y_{1,256128}
\\ y_{2,1} & y_{2,2} & \ldots & y_{2,256128}
\\ \vdots & \vdots & \ddots & \vdots
\\ y_{n,1} & y_{n,2} & \ldots & y_{n,256128}
\end{bmatrix}$

Dimensiones:

- $ X \in \mathbb{R}^{n \times 1152} $

- $ W \in \mathbb{R}^{256128 \times 1152} $

- $ Y \in \mathbb{R}^{n \times 256128} $

Algunas líneas relevantes del código fuente:

```python
return jnp.dot(x, self.input_embedding_table.T)
```

### 4.3. Sampling

Para convertir la matriz de _logits_ en _tokens_ se utiliza una estrategia de _sampling_. Por ejemplo, una estrategia _greedy_ buscará simplemente el _token_ del vocabulario con el _logit_ (puntuación) más alta. Otras estrategias pueden utilizar criterios más complejos y añadir parámetros como la temperatura para seleccionar los _tokens_.

De igual forma que el _tokenizer_ se aplica sobre los datos en crudo antes de introducirlos en el modelo, la estrategia de _sampling_ se aplica sobre los datos en crudo retornados por el modelo. Ninguno de los dos componentes pertenecen al modelo, pero se comentan por completitud en el análisis.

El sistema es autoregresivo, lo que quiere decir que la salida alimenta su propia entrada.

```
Input ─> Embedding ─> 26 x Decoder ─> Norm ─> Logits -> Output
      ↑                                                   │
      └───────────────────────────────────────────────────┘
```

Empieza con un _token_, tradicionalmente el _BOS_ (_Begin of Sentence_), genera un nuevo _token_, y lo inyecta de vuelta en el modelo. De esta forma la secuencia de salida va creciendo a cada iteración. El proceso se detiene al generar un _token_ determinado, tradicionalmente el _EOS_ (_End Of Sentence_), o al alcanzar un número máximo prefijado de _tokens_ de salida.